In [1]:
pip install accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import json
from datasets import Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

def preprocess_example(example):
    example_dict = {
        "srno": example.get("srno", None),
        "nl_command": example.get("nl_command", None),
        "bash_code": example.get("bash_code", None),
    }
    text = f"[INST] Docstring: {example_dict['nl_command']} [/INST] Code: {example_dict['bash_code']}"
    return {"text": text}

def load_json_as_dataset(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    return Dataset.from_list(data)

def finetune_llama_v2():
    # Load your dataset
    train_dataset = load_json_as_dataset("/content/drive/MyDrive/majorproj/data_train.json")

    # Preprocess the dataset
    train_data = train_dataset.map(preprocess_example, remove_columns=["srno", "nl_command", "bash_code"])

    tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
    tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        "codellama/CodeLlama-7b-hf", quantization_config=bnb_config, device_map="auto"
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

    training_arguments = TrainingArguments(
        output_dir="codellama2-finetuned-nl2bash",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=500,
        fp16=True,
        push_to_hub=False,
        report_to="none"
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=train_data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )

    trainer.train()
    return model, tokenizer

if __name__ == "__main__":
    ft_mode, og_tokenizer = finetune_llama_v2()

Map:   0%|          | 0/19658 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/19658 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,2.347900
20,1.555200
30,1.365500
40,1.298800
50,1.243400
60,1.249900
70,1.218000
80,1.135200
90,1.126700
100,1.173000


Step,Training Loss
10,2.347900
20,1.555200
30,1.365500
40,1.298800
50,1.243400
60,1.249900
70,1.218000
80,1.135200
90,1.126700
100,1.173000


In [10]:
ft_model = ft_mode
def run_inference(input_text):

    # Tokenize the user input
    input_ids = og_tokenizer(input_text, return_tensors="pt").input_ids

    # Feed the tokenized input into the model for inference
    output_ids = ft_model.generate(input_ids, max_length=200, num_return_sequences=1, temperature=0.7)

    # Decode the output tokens to generate the predicted bash command
    output_text = og_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text

if __name__ == "__main__":
    # Example usage
    nl_command = 'List all the processes that are using more than 100MB of memory and sort them by memory usage in descending order.'

    # Prepare the input for the model
    input_text = f"[INST] Docstring: {nl_command} [/INST] Code:"
    predicted_bash_command = run_inference(input_text)
    print("Predicted Bash Command:", predicted_bash_command)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted Bash Command: [INST] Docstring: List all the processes that are using more than 100MB of memory and sort them by memory usage in descending order. [/INST] Code: ps aux | awk '{if ($3 > 100) print $0}' | sort -k 4 -r | head -n 10 | nl -nln -s ' ' | sed 's/^[ \t]*//' | column -t -s ' ' | less -S -N -X -K -s -m -p '^[0-9]*' -e '/\s*$/d' -e '/\s*$/d' -e '/\s*$/d' -e '/\s*$/d' -e '/\s*$/d' -e '/\s*$/d' -e '/\s*$/d' -e '/\s*$/d' -


In [11]:
# Define the natural language command
nl_command = 'Bash code to list all the processes that are using more than 100MB of memory and sort them by memory usage in descending order.'

# Prepare the input for the model
input_text = f"[INST] Docstring: {nl_command} [/INST] Code:"

# Tokenize the input
inputs = og_tokenizer(nl_command, return_tensors="pt").to(ft_model.device)

# Generate the output
outputs = ft_model.generate(**inputs, max_length=150)

# Decode the output
generated_code = og_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Bash Command:")
print(generated_code)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Bash Command:
Bash code to list all the processes that are using more than 100MB of memory and sort them by memory usage in descending order.

\begin{code}
ps -eo pid,user,group,rss,vsz,args | awk '$5>100000 {print $0}' | sort -k5nr
\end{code}

Comment: I think you need to add a `,` after `vsz` in the `ps` command.

Comment: @Kusalananda, yes, you are right. Thanks for pointing it out.

Answer: \begin{code}
ps -eo pid,user


In [15]:
test_dataset = load_json_as_dataset("/content/drive/MyDrive/majorproj/data_test.json")
test_data = test_dataset.map(preprocess_example, remove_columns=["srno", "nl_command", "bash_code"])


Map:   0%|          | 0/2458 [00:00<?, ? examples/s]

In [16]:
sample_example = test_data[1976]

import re

# Input string
input_string = sample_example['text']

# Regular expression pattern to extract instruction and code
pattern = r'\[INST\] Docstring: (.+?) \[/INST\] Code: (.+)'

# Match the pattern
match = re.match(pattern, input_string)

# Extract instruction and code
if match:
    nl_command = match.group(1)
    actual_code = match.group(2)
    print("nl_command =", nl_command)
    print("code =", actual_code)
else:
    print("No match found.")

nl_command = Find all .mp3 files with more then 10MB and delete them from root directory .
code = find /  -type f -name *.mp3 -size +10M -exec rm  {} \;


In [17]:
input_text = f"Bash code for {nl_command}"
inputs = og_tokenizer(input_text, return_tensors="pt").to(ft_model.device)
outputs = ft_model.generate(**inputs, max_length=150)
generated_code = og_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the results
print("Input NL Command:", nl_command)
print("Predicted Bash Command:", generated_code)
print(f"actual_code: {actual_code}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input NL Command: Find all .mp3 files with more then 10MB and delete them from root directory .
Predicted Bash Command: Bash code for Find all .mp3 files with more then 10MB and delete them from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .mp3 files are deleted from root directory .
actual_code: find /  -type f -name *.mp3 -size +10M -exec rm  {} \;


In [ ]:
import torch
torch.save(ft_model.state_dict(), '/content/drive/MyDrive/majorproj/codellama-fine-tuned-nl2bash')